In [16]:
#Import Packages
import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression
import string
import statsmodels.api as sm
import pickle
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import pysolar.solar as solar
from geographiclib.geodesic import Geodesic
import xarray as xr
import pytz
import simplekml
from pylr2 import regress2
import cartopy
import sklearn
import sys
sys.path.append('..')
import funcs.ac_funcs as ac
import funcs.plotting as plotting

pd.options.mode.chained_assignment = None

%load_ext autoreload
%autoreload 2


# Caltech met data

In [38]:
# separate_caltech_weather_path = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/met/caltech' 
# met_save_path = '/uufs/chpc.utah.edu/common/home/lin-group15/agm/em27/ua/inst_data/met/caltech/'

# dates = ['2023-05-16','2023-05-17','2023-05-18','2023-05-19']
# for date in dates:
#     dfs = []
#     for file in sorted(os.listdir(separate_caltech_weather_path)):
#         if file.startswith(date):
#             df = pd.read_csv(os.path.join(separate_caltech_weather_path,file))
#             df['dt'] = pd.to_datetime(df['Time'])
#             df = df.set_index('dt')
#             df = df.drop('Time',axis = 1)
#             dfs.append(df)# = pd.concat([date_df,df])
#     date_df = pd.concat(dfs,axis = 1)
#     date_df = date_df.resample('1T').mean().interpolate(method='linear')
#     date_df = date_df.rename(columns = {'Relative Humidity (%)':'RH','Pressure (mb)':'Pout','Temperature (&deg;C)':'Tout'})
#     date_df['UTCDate'] = date_df.index.strftime('%y/%m/%d')
#     date_df['UTCTime'] = date_df.index.strftime('%H:%M:%S')
#     date_df = date_df.reset_index(drop=True)[['UTCDate','UTCTime','Tout','Pout','RH']]
#     save_name = f"{date.replace('-','')}_caltech.txt"
#     date_df.to_csv(os.path.join(met_save_path,save_name),index=False)


# TCCON/EM27 Comparison 

In [39]:
#Load the EM27 Data
em27_data_folder = "/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/EM27_oof/summer_2023/aaron_retrievals/ua/"

dt1_str = '2023-05-16 00:00:00'
dt2_str =  '2023-05-19 10:00:00'
tz = 'UTC'

oof_manage = ac.oof_manager(em27_data_folder,tz)
em27_df = oof_manage.load_oof_df_inrange(dt1_str,dt2_str,True)

em27_df_sub = em27_df[['xch4(ppm)','xch4(ppm)_error','xco2(ppm)','xco2(ppm)_error','xco(ppb)','xco(ppb)_error']]
for col in em27_df_sub.columns:
    new_col_name = col.replace('(ppm)','')
    new_col_name = new_col_name.replace('(ppb)','')
    new_col_name = new_col_name+'_em27'
    em27_df_sub = em27_df_sub.rename(columns={col:new_col_name})

In [66]:
## Load the TCCON data into a dataframe

# Point to the TCCON netcdf we want to read
tccon_data_fullpath = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/TCCON/ci20230502_20230701.private.nc'
f = netCDF4.Dataset(tccon_data_fullpath)

# format the time 
time = f.variables['time']
datetimes = netCDF4.num2date(time[:], time.units)
datetimes = [date.strftime('%Y-%m-%d %H:%M:%S') for date in datetimes]

# list the variables you want to keep
cols = ['lat', 'long','xco2','xco2_error','xco','xco_error','xch4','xch4_error']

# extract those variables from the netcdf and append together
list_nc = []
for c in cols:
    list_nc.append(list(f.variables[c][:]))
    
# convert into pandas dataframe
tccon_df = pd.DataFrame(list_nc)
tccon_df = tccon_df.T # transpose the dataframe
tccon_df.columns = cols # set column names

# add the time as the dataframe index 
tccon_df.index = pd.to_datetime(datetimes, utc = True)

#trim the tccon df to the range of data in the em27 df
tccon_df_sub = tccon_df.loc[(tccon_df.index>=min(em27_df.index))&(tccon_df.index<=max(em27_df.index))]
tccon_df_sub = tccon_df_sub[['xch4','xch4_error','xco2','xco2_error','xco','xco_error']]
for col in tccon_df_sub.columns:
    new_col_name = col+'_tccon'
    tccon_df_sub = tccon_df_sub.rename(columns={col:new_col_name})

In [67]:
resample_interval = '1T'
clip_range = ['2023-05-16 20:00:00','2023-05-19 00:00:00']
em27_df_resampled = em27_df_sub.resample(resample_interval).mean(numeric_only=True)
tccon_df_resampled = tccon_df_sub.resample(resample_interval).mean(numeric_only=True)
merged_df = pd.concat([em27_df_resampled,tccon_df_resampled],axis = 1)
merged_df = merged_df.loc[(merged_df.index>=clip_range[0])&(merged_df.index<=clip_range[1])]
merged_df = merged_df.dropna()

In [68]:
species = ['xch4','xco2','xco']
fig = make_subplots(rows=3,cols=1)
row = 1
for spec in species:
    if row == 1:
        showlegend = True
    else:
        showlegend = False
    fig.add_trace(go.Scatter(
        x = merged_df.index,
        y = merged_df[f'{spec}_em27'],
        mode = 'markers',
        marker = {'color' : 'grey'},
        name = 'em27',
        showlegend = showlegend
    ), row = row , col = 1)
    fig.add_trace(go.Scatter(
        x = merged_df.index,
        y = merged_df[f'{spec}_tccon'],
        mode = 'markers',
        marker = {'color' : 'red'},
        name = 'tccon',
        showlegend = showlegend
    ), row = row , col = 1)
    fig.update_yaxes(title_text=f'{spec}',row=row,col=1)
    row += 1

fig.update_layout(
    height=600,
    margin = go.layout.Margin(t=1,b=1)
)